In [2]:
import pandas as pd
import numpy as np
import verispy, os
from verispy import VERIS

[Source for VERISPY + Useful README](https://github.com/RiskLens/verispy)

In [67]:


DATA_DIR = "./data/validated/"
PKL_PATH = "./data/vdf.pkl"

v = VERIS(json_dir=DATA_DIR, verbose=True)
# json_to_df is an expensive function, so we save the converted dataframe for use between sessions. 
if not os.path.exists(PKL_PATH):
    print("we're making pickles")
    vdf = v.json_to_df()
    vdf.to_pickle(PKL_PATH)


vdf = pd.read_pickle(PKL_PATH)
# print(vdf.dtypes)
# print(vdf['victim.state'])
v.enum_summary(vdf, "action")


Found 8323 json files.


,enum,x,n,freq
0,Error,2418,8105.0,0.29833
1,Hacking,2213,8105.0,0.27304
2,Misuse,1697,8105.0,0.20938
3,Physical,1576,8105.0,0.19445
4,Malware,697,8105.0,0.08600
5,Social,568,8105.0,0.07008
6,Environmental,8,8105.0,0.00099
7,Unknown,218,NaN,NaN


##### v is a VERIS object. vdf contains the actual data. 

[Good StackOverflow Answer on working with Dataframes vvv](https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values)

In [78]:
df_impact = vdf.loc[:, vdf.columns.str.contains('rating')]
print(df_impact.columns)
print(v.enum_summary(vdf, 'impact.overall_rating'))
print(v.enum_summary(vdf, "impact.loss.rating"))
# v.enum_summary(vdf, "impact.rating")
# vdf
# v.enum_summary(vdf, "impact.rating"

Index(['impact.loss.amount.Operating costs', 'impact.loss.rating.Major',
       'impact.loss.rating.Minor', 'impact.loss.rating.Moderate',
       'impact.loss.rating.None', 'impact.loss.rating.Unknown',
       'impact.loss.variety.Operating costs',
       'impact.overall_rating.Catastrophic', 'impact.overall_rating.Damaging',
       'impact.overall_rating.Distracting',
       'impact.overall_rating.Insignificant', 'impact.overall_rating.Painful',
       'impact.overall_rating.Unknown'],
      dtype='object')
            enum     x      n     freq
0  Insignificant    52  116.0  0.44828
1    Distracting    25  116.0  0.21552
2       Damaging    20  116.0  0.17241
3        Painful    18  116.0  0.15517
4   Catastrophic     1  116.0  0.00862
5        Unknown  5767    NaN      NaN
       enum  x    n  freq
0     Major  0  0.0   NaN
1     Minor  0  0.0   NaN
2  Moderate  0  0.0   NaN
3      None  0  0.0   NaN
4   Unknown  0  NaN   NaN


In [69]:
df_action = vdf.loc[:, vdf.columns.str.startswith('action')]

In [11]:
df_final = df_action.iloc[:, :7]

In [12]:
df_bugs = df_final[df_final['action.Hacking'] | df_final['action.Malware']]


In [13]:
indices = list(df_bugs.index)
bugs = [False] * df_final.shape[0]
for i in indices:
    bugs[i] = True

In [14]:
df_final['action.Bugs'] = bugs

In [15]:
df_final

,action.Environmental,action.Error,action.Hacking,action.Malware,action.Misuse,action.Physical,action.Social,action.Bugs
0,False,False,False,False,True,False,False,False
1,False,False,False,False,False,True,False,False
2,False,True,False,False,False,False,False,False
3,False,False,True,False,False,False,False,True
4,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
8318,False,True,False,False,False,False,False,False
8319,False,False,False,False,True,False,False,False
8320,False,False,False,False,True,False,False,False
8321,False,False,False,False,False,True,False,False


In [16]:

N = 1000
sample = np.random.choice(8323, N)
df_short = df_final.iloc[sample,:].reset_index().iloc[:,1:]

In [17]:
df_short

,action.Environmental,action.Error,action.Hacking,action.Malware,action.Misuse,action.Physical,action.Social,action.Bugs
0,False,True,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False
2,False,False,True,False,False,False,False,True
3,False,True,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...
995,False,False,False,False,False,True,False,False
996,False,False,False,False,True,False,False,False
997,False,False,True,False,False,False,False,True
998,False,False,False,False,True,False,False,False


In [18]:
df_impact = 

SyntaxError: invalid syntax (<ipython-input-18-d56ff05de301>, line 1)